# Electricity Price Forecasting on the German day-ahead market

This notebook is the main interface to the associated EPF library. Various parameters can be adjusted within the library via the Config file. A more detailed description of the individual parameters can be found within the configuration classes. In the “exploratory_analysis” notebook, the raw data sets are analyzed exploratively and the results are displayed visually. Based on these results, features from the data sets were specifically selected for further use in the deep learning pipeline.

The forecasting pipeline is built to automatically perform data preprocessing including data cleaning, outlier removal and seasonal decomposition. Within the configuration feature engineering can be toggled on and of for each feature. Forecasting is done with two different models, that can be retrained and saved anytime. The models available are an LSTM and GRU. They all perform multi-step ahead, single shot forecasts. This means by default each forecast contains 24 time steps into the future, that are forecasted together in one computation.

In [1]:
from epf.pipeline import EpfPipeline
import pickle as pkl

pipeline = EpfPipeline()

2025-05-15 15:23:40.840 | INFO     | epf.config:<module>:19 - PROJ_ROOT path is: C:\Users\valen\PycharmProjects\epf
2025-05-15 15:23:40.841 | INFO     | epf.config:create_dir:15 - DATA_DIR path is: C:/Users/valen/PycharmProjects/epf/data
2025-05-15 15:23:40.842 | INFO     | epf.config:create_dir:15 - RAW_DATA_DIR path is: C:/Users/valen/PycharmProjects/epf/data/raw
2025-05-15 15:23:40.843 | INFO     | epf.config:create_dir:15 - INTERIM_DATA_DIR path is: C:/Users/valen/PycharmProjects/epf/data/interim
2025-05-15 15:23:40.844 | INFO     | epf.config:create_dir:15 - PROCESSED_DATA_DIR path is: C:/Users/valen/PycharmProjects/epf/data/processed
2025-05-15 15:23:40.844 | INFO     | epf.config:create_dir:15 - TRAIN_DATA_DIR path is: C:/Users/valen/PycharmProjects/epf/data/processed/train_data
2025-05-15 15:23:40.845 | INFO     | epf.config:create_dir:15 - MODELS_DIR path is: C:/Users/valen/PycharmProjects/epf/models
2025-05-15 15:23:40.846 | INFO     | epf.config:create_dir:15 - PREDICTIONS_D

Pipeline initialized with:
FeatureConfig: 
Selected features: 
['DE-LU Prices', 'CH Prices', 'DK1 Prices', 'DK2 Prices', 'FR Prices', 'DE Solar Generation', 'DE Wind Generation Offshore', 'DE Wind Generation Onshore', 'DE Load', 'Month', 'Day of Week', 'Holiday']
WINDOW_LENGTH: 24
N_SIGMA: 3
METHOD: nearest

ModelConfig: 
TRAIN_SPLIT (upper bound): 2023-09-30 22:00:00+00:00
VALIDATION_SPLIT (upper bound): 2023-12-31 22:00:00+00:00
MAX_EPOCHS: 100
OUT_STEPS: 24
SEASONALITY_PERIOD: 24
INPUT_WIDTH_FACTOR: 1.25
MODEL_BUILDER: GRU
USE_HIDDEN_LAYERS: True
NUM_FEATURES: 12
UNIT_MIN_VALUE: 32
UNIT_MAX_VALUE: 128
UNIT_STEP: 32
LR_MIN_VALUE: 0.0001
LR_MAX_VALUE: 0.1
LR_STEP: 0.001
DROPOUT_RATE_MIN_VALUE: 0.2
DROPOUT_RATE_MAX_VALUE: 0.7
DROPOUT_RATE_STEP: 0.05
NUM_LAYERS_MIN: 1
NUM_LAYERS_MAX: 3
NUM_LAYERS_STEP: 1
MAX_TRIALS: 50
LABEL_COL: de_prices_hat_rm_seasonal




In [ ]:
model_name = 'gru_vre_neighboring_prices'

pipeline.train(model_name, overwrite=True)
pipeline.evaluate(model_name)

In [ ]:
import pickle as pkl

with open("../models/gru_all_features.pkl", "rb") as f:
    model_obj = pkl.load(f)

window = model_obj['window']
model = model_obj['best_model']
model_name = model_obj['model_name']
test_df = model_obj['test_df']
plot_col = 'de_prices_hat_rm_seasonal'

window.plot(model, plot_col=plot_col)

In [ ]:
hp = model_obj['best_hps']
hp.values

In [ ]:
model.summary()